In [1]:
from mxnet import nd 
from mxnet.gluon import nn 
net=nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(256,activation='relu'))
    net.add(nn.Dense(10))
print(net)

Sequential(
  (0): Dense(None -> 256, Activation(relu))
  (1): Dense(None -> 10, linear)
)


# 使用nn.Block来定义 
事实上，nn.Sequential 是nn.Block 的简单形式。我们来看下如果使用nn.Block来实现相同的网络。 


In [2]:
class MLP(nn.Block):
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        with self.name_scope():
            self.dense0=nn.Dense(256)
            self.dense1=nn.Dense(10)
    def forward(self,x):
        return self.dense1(nd.relu(self.dense0(x)))

可以看到nn.block的使用是通过创建一个它子类的类，其中至少包含了两个函数。
__init__:创建参数，上面的例子我们使用了包含了参数的dense 层 
forward(): 定义网络的计算 

我们所创建的类的使用跟前面net 没有太多不一样 。 

In [3]:
net2=MLP()
print(net2)
net2.initialize()
x=nd.random.uniform(shape=(4,20))
y=net2(x) 
y

MLP(
  (dense0): Dense(None -> 256, linear)
  (dense1): Dense(None -> 10, linear)
)



[[ 0.03126615  0.04562764  0.00039857 -0.08772386 -0.05355632  0.02904574
   0.08102557 -0.01433946 -0.04224151  0.06047882]
 [ 0.02871901  0.03652265  0.00630051 -0.05650971 -0.07189322  0.08615957
   0.05951559 -0.06045965 -0.0299026   0.05651001]
 [ 0.02147349  0.04818896  0.05321142 -0.12616856 -0.0685023   0.09096345
   0.04064304 -0.05064794 -0.02200242  0.04859561]
 [ 0.03780478  0.0751239   0.03290457 -0.11641113 -0.03254967  0.0586529
   0.02542157 -0.01697343 -0.00049652  0.05892839]]
<NDArray 4x10 @cpu(0)>

In [4]:
print("default prefix",net2.dense0.name) 
net3=MLP(prefix='another_mlp_')
print('customized prefix:',net3.dense0.name)

default prefix mlp0_dense0
customized prefix: another_mlp_dense0


In [5]:
class Sequential(nn.Block):
    def __init__(self,**kwargs):
        super(Sequential,self).__init__(**kwargs)
    def add(self,block):
        self._children.append(block)
    def forward(self,x):
        for block in self._children:
            x=block(x)
        return x 

可以跟nn.Sequential 一样的使用这个自定义的类；

In [6]:
net4=Sequential()
with net4.name_scope():
    net4.add(nn.Dense(256,activation='relu'))
    net4.add(nn.Dense(10))

net4.initialize()
y=net4(x) 
y 

AttributeError: 'collections.OrderedDict' object has no attribute 'append'